<a href="https://colab.research.google.com/github/samkhur006/LLM-based-models/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import math


In [2]:
class InputEmbeddings(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super().__init__()
    self.d_model=d_model
    self.vocab_size =vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    return self.embedding(x) * math.sqrt(self.d_model)

In [3]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model: int, seq_len: int, dropout: float):
    super().__init__()
    self.d_model=d_model
    self.seq_len=seq_len
    self.dropout= nn.Dropout(dropout)

    pe = torch.zeros(seq_len, d_model)
    position = torch.arange(0,seq_len,dtype=torch.float).unsqueeze(1)    # (Seq_len, 1 )

    div_term = torch.exp ( -math.log(10000)*torch.arange(0,d_model,2)/d_model   )

    pe[:,0::2]=torch.sin(position * div_term)
    pe[:,1::2]=torch.cos(position * div_term)

    pe=pe.unsqueeze(0) #(1,Seq_len,d_model)

    self.register_buffer('pe',pe)
  def forward(self, x):
    x= x + (pe[:,:x.shape[1],:]).requires_grad_(False)
    return self.dropout(x)


In [4]:
class LayerNormalization(nn.Module):
  def __init__(self, eps:float = 10**-6):
    super().__init__()
    self.eps=eps
    self.alpha=nn.Parameter(torch.ones(1))
    self.bias=nn.Parameter(torch.zeros(1))
  def forward(self,x):
    mean = x.mean(dim=-1,keepdim=True)
    std=x.std(dim=-1, keepdim=True)
    return self.alpha * (x-mean)/(std + self.eps )   + self.bias

In [5]:
class FeedForwardLayer(nn.Module):
  def __init__(self, d_model: int , d_hidden: int , dropout: float):
    super().__init__()
    self.layer1 = nn.Linear(d_model,d_hidden)
    self.relu=torch.ReLU()
    self.layer2 = nn.Linear(d_hidden, d_model)
    self.dropout=nn.Dropout(dropout)
  def forward(self, x):
    return self.layer2(self.dropout (self.relu(self.layer1(x)) ) )

In [6]:
class MultiHeadAttention(nn.Module):
  def _init__(self, d_model: int, h: int, dropout: float):
    super().__init__()
    self.d_model=d_model
    self.h=h;
    assert d_model %h ==0
    self.dropout= dropout
    self.d_k = d_model //h
    self.w_q = nn.Linear(d_model,d_model)
    self.w_k = nn.Linear(d_model,d_model)
    self.w_v = nn.Linear(d_model,d_model)

    self.w_o=nn.Linear(d_model,d_model)  # in trivial case
    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout):
    d_k=query.shape[-1]
    attention_scores = (query @ key.transpose(-2,-1)) / math.sqrt(d_k)             ################## Matrix mult  (Batch, h , seq_len, seq_len)

    if mask is not None:
      attention_scores.masked_fill_(mask ==0, -1e9)
    attention_scores = attention_scores.softmax(dim=-1)

    if dropout is not None:
      attention_scores = dropout(attention_scores)

    return (attention_scores @ value), attention_scores

  def forward(self, q,k,v, mask ):       # q=k=v (self attention)
     query = self.w_q(q)    # (Batch, seq_len, d_model)
     key = self.w_q(k)
     value = self.w_q(v)

     query =query.view(query.shape[0],query.shape[1],self.h ,self.d_k).transpose(1,2)       #(Batch, h, seq_len, d_k)
     key =key.view(key.shape[0],key.shape[1],self.h ,self.d_k).transpose(1,2)
     value =value.view(value.shape[0],value.shape[1],self.h ,self.d_k).transpose(1,2)

     x, self.attention_scores  = MultiHeadAttention.attention(query,key,value,mask,self.dropout)

     x=x.transpose(1,2).contiguous().view(x.shape[0],-1,self.h * self.d_k)

     return self.w_o(x)


In [7]:
class ResidualConnection(nn.Module):
  def __init__(self, dropout: nn.Dropout):
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = LayerNormalization()


  def forward(self, x, sublayer):           #Sublayer is previous layer
    return x + self.dropout(sublayer(  self.norm(x)  )  )

In [8]:
class EncoderBlock(nn.Module):
  def __init__(self, self_attention_block: MultiHeadAttention, feed_forward_block: FeedForwardLayer, dropout:float ):
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections= nn.ModuleList( [ResidualConnection(dropout) for _ in range(2)]  )

  def forward(self,x, src_mask):              ##### src_mask to prevent interaction of padding words with other words
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x,x,x,src_mask) )
    x = self.residual_connections[1](x,  self.feed_forward_block )
    return x



In [9]:
class Encoder(nn.Module):
  def __init__(self, layers: nn.ModuleList):
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()

  def forward(self, x, mask):
    for layer in self.layers:
      x=layer(x,mask)
    return self.norm(x)

In [10]:
class DecoderBlock(nn.Module):
  def __init__(self, self_attention_block: MultiHeadAttention, cross_attention_block: MultiHeadAttention, feed_forward_block: FeedForwardLayer, dropout: float):
    super().__init__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections= nn.ModuleList( [ResidualConnection(dropout) for _ in range(3)]  )

  def forward(self,x, encoder_output, src_mask, tgt_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x,x,x,tgt_mask) )
    x = self.residual_connections[1](x, lambda x: self.self_attention_block(x,encoder_output,encoder_output,src_mask) )        ######### WTF why src_mask?
    x = self.residual_connections[2](x,  self.feed_forward_block )
    return x


In [11]:
class Decoder(nn.Module):
  def __init__(self, layers: nn.ModuleList):
    super().__init__()
    self.layers=layers
    self.norm=LayerNormalization()

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    for layer in self.layers:
      x=layer(x, encoder_output, src_mask, tgt_mask)
    return self.norm(x)

In [12]:
class ProjectionLayer(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super().__int__()
    self.proj = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    return torch.log_softmax(self.proj(x), dim=-1)

In [13]:
class Transformer(nn.Module):
  def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

  def encode(self, src, src_mask):
    src = self.src_embed(src)
    src = self.src_pos(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output, src_mask, tgt, tgt_mask):
    tgt = self.tgt_embed(tgt)
    tgt= self.tgt_pos(tgt)
    return self.decoder(tgt, encoder_output, src_mask, tgt_mask )

  def project(self,x):
    return self.projection_layer(x)


In [17]:
from torch.nn.modules.activation import MultiheadAttention
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int =512, N: int =6, h:int = 8, dropout: float = 0.1, d_ff :int =2048):
  #Create embedding layer
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    encoder_blocks = []

    for _ in range(N):
      encoder_self_attention_block = MultiheadAttention(d_model, h, dropout)
      feed_forward_block = FeedForwardLayer(d_model, d_ff, dropout)
      encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
      encoder_blocks.append(encoder_block)


    decoder_blocks = []

    for _ in range(N):
      decoder_self_attention_block = MultiheadAttention(d_model, h, dropout)
      decoder_cross_attention_block = MultiheadAttention(d_model, h, dropout)
      feed_forward_block = FeedForwardLayer(d_model, d_ff, dropout)
      decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
      decoder_blocks.append(decoder_block)


    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # initialize params

    for p in transformer.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)

    return transformer


